In [7]:
import pandas as pd
from sklearn.externals import joblib
import category_encoders as ce
import sys
import os
# BinEncoder = joblib.load('BinEncoder.pkl') 
clf = joblib.load('RF_generos.pkl')

YearBinaryEnco = joblib.load('YearBinaryEnco.pkl')
tfidf_plot = joblib.load('tfidf_plot.pkl')
tfidf_title = joblib.load('tfidf_title.pkl')


def clasff_movie(title, plot, year):
    
    
    #Preprocess
    title_clean = text_clean(title, remove_stop_words=False)
    plot_clean  = text_clean(plot)
    
    #Transform
    YearBinary = YearBinaryEnco.transform(year)
    title_tfidf_dtm = tfidf_title.transform(title_clean)
    plot_tfidf_dtm = tfidf_plot.transform(plot_clean)
    
    #Create a dataframe 
    df_ = pd.concat([plot_feat_tfidf.add_suffix('_1'), 
                      title_feat_tfidf.add_suffix('_2'), 
                      YearBinary], axis=1) 
    
    predict_ = rf.predict(df_)

    return predict_




# This function transform the text in order get ready data, remove stop words, stimming, Lemmatisation and n_grams
def text_clean(text, remove_stop_words=True):
    wordnet_lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer('english')
    document = text
    
    # Remove all the special characters
    document = re.sub(r'\W', ' ', document)

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    #Removing punctuation
    document = re.sub(r'[^\w\s]', '', document)

    #LowerCase    
    document = document.lower()
    
    #Split document word a word
    words_document = text.split()
    
    #Remove clean_words
    words_document = [word for word in words_document if word not in waste_words]
    #Remove stop words
    if remove_stop_words:
        words_document = [word for word in words_document if word not in custom_stopwords]
    
    #stimming
    words_document = [stemmer.stem(word) for word in words_document]
    
    #Lemmatisation
    words_document = [wordnet_lemmatizer.lemmatize(word) for word in words_document]
    words_document = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words_document]
           
    return ' '.join(words_document)


FileNotFoundError: [Errno 2] No such file or directory: 'RF_generos.pkl'

In [5]:
#!/usr/bin/python
from flask import Flask
from flask_restplus import Api, Resource, fields
from sklearn.externals import joblib
from Model_Movie_CLF import clasff_movie

app = Flask(__name__)

api = Api(
    app, 
    version='1.0', 
    title='Movie Genre Classification',
    description='Desarrollado por: Iván Gómez, Cristian Najera, Natalia Martínez')

ns = api.namespace('Classification', 
     description='Classification movie')
   
parser = api.parser()

parser.add_argument(
    'Title', 
    type=str, 
    required=True, 
    help='Title of the movie', 
    location='args')

parser.add_argument(
    'Plot', 
    type=str, 
    required=True, 
    help='Description of the movie', 
    location='args')

parser.add_argument(
    'Year', 
    type=str, 
    required=True, 
    help='Year of the movie', 
    location='args')


resource_fields = api.model('Resource', {
    'result': fields.String,
})

@ns.route('/')
class PredPriceApi(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        return {
         "result": clasff_movie(args['Title'],args['Plot'],args['Year'])
        }, 200
    
    
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=8889)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:8889/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Apr/2019 14:26:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2019 14:26:44] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2019 14:27:24] "GET /Classification/?Title=Este%20es%20title&Plot=Este%20es%20title%20plot&Year=2009 HTTP/1.1" 200 -
127.0.0.1 - - [21/Apr/2019 14:28:19] "GET /Classification/?Title=Este%20es%20title&Plot=in%20sweden%20%2C%20%20a%20female%20blackmailer%20with%20a%20disfiguring%20facial%20scar%20meets%20a%20gentleman%20who%20lives%20beyond%20his%20means%20.%20%20they%20become%20accomplices%20in%20blackmail%20%2C%20%20and%20she%20falls%20in%20love%20with%20him%20%2C%20%20bitterly%20resigned%20to%20the%20impossibility%20of%20his%20returning%20her%20affection%20.%20%20her%20life%20changes%20when%20one%20of%20her%20victims%20proves%20to%20be%20the%20wife%20of%20a%20plastic%20surgeon%20%2C%20%20who%20catches%20her%20in%20his%20apartment%20%2C%20%20but%20believes%20her%20to%20be%20a%2